# Multirate sampling concept
This notebook shows the concepts behind multirate sampling.  It is to aid in understanding, and does not represent a practical approach to re-sampling data.  The notebook allows you to experiment with changing the sampling factors, as well as changing the number of original data samples.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters

The following parameters can be specified.  

Parameter | Meaning
--------- | -------
<code>orig_samples</code> | The number of samples in the original sequence (e.g. 8)
<code>I</code>, <code>D</code> | Integers specifying the resampling rate as fs * I / D (e.g. 2, 3)
<code>FFT_length</code> |Length of the FFT after zero padding the impulse response (e.g. 2048)

In [ ]:
orig_samples = 8
I = 2
D = 3

### Generate data
Random values are used to define the samples being used.  In order to avoid the need to worry about filtering when moving to a lower sampling rate, implement a brickwall filter in the frequency domain on a long sequence of values.  We will then select only the first 8 values.

In [ ]:
long_seq = np.random.randn(orig_samples * 8)*2 + 2
if D>I:
    long_seq_freq = np.fft.fft(long_seq)
    cut_point = len(long_seq) * I // (2 * D)
    long_seq_freq[cut_point:len(long_seq_freq)-cut_point] = 0
    long_seq = np.fft.ifft(long_seq_freq).real
data = long_seq[:orig_samples]

# Define the timepoints for the original data, and the re-sampled data
data_timepoints = np.arange(0, orig_samples)
resampled_timepoints = np.arange(0, orig_samples, step = D/I)

Define x-scale for the figures

In [ ]:
x_range = np.linspace(min(data_timepoints)-0.25,
                      max(data_timepoints)+0.75, 3201)

### Define plotting function
This is the main part of the code.  The first part displays the samples at the original sampling rate.  After that, those samples are each convolved with a sinc function, and then combined together to generate samples at the re-sampled rate, as well as at a very high sampling rate to represent the analogue reconstructed signal.  Depending upon the plot selected, different elements are displayed.

In [ ]:
def plot_fig(x, y, Sincs, Sum, name):
    """
       Plot the amplitude of sample.
    
       INPUT:
           x (array-like): The x-positions of the stems. 
           y (array-like): The y-values of the stem heads.
           Sincs   (bool): True if displaying sinc functions
           Sum     (bool): True if displaying reconstructed sampling data.
           name  (string): The name used to save figure.
           
    """
    ### Display original samples
    plt.figure(figsize = (16, 8))
    plt.rcParams.update({'font.size': 16})
    
    # Plot a baseline y = 0
    plt.axhline(y = 0, color = 'black')
    
    # Plot the original data samples
    (markerLines, stemLines, baseLines) = plt.stem(x, y, use_line_collection = True)
    markerLines.set_markerfacecolor('none')
    plt.setp(baseLines, color = 'black')
    
    ### Construct high rate samples, and re-sampled data
    # Initialise the accumulated results
    sums = np.zeros(x_range.size)
    resampled_data = np.zeros(len(resampled_timepoints))
    
    # For each data sample, determine its contribution to the reconstructed signal
    for index in range(0, len(data)):
        resampled_data = (resampled_data + data[index] * 
                          np.sinc(resampled_timepoints - data_timepoints[index]))
        if Sincs:
            # Display the scaled sinc function
            plt.plot(x_range, data[index]*np.sinc(x_range-data_timepoints[index]))
        if Sum:
            # Combine with previous sample contributions
            sums += data[index]*np.sinc(x_range-data_timepoints[index])
    
    ### Display re-sampled data if we are not showing the sinc functions
    if not Sincs:
        # Make original samples lighter
        plt.setp(stemLines, color = [0.75, 0.75, 0.75], linewidth=1)
        # Plot the re-sampled data points
        (markerLines, stemLines, baseLines) = plt.stem(resampled_timepoints,
                                                       resampled_data,
                                                       use_line_collection = True)
        plt.setp(baseLines, color = 'black', linewidth=1)
        
    ### Display "reconstructed analogue signal" (resampling with a high sampling rate)
    if Sum:
        # Display the reconstructed signal
        plt.plot(x_range, sums)
        
    plt.xlim([x_range[0], x_range[-1]])
    plt.xlabel('Sample')
    plt.ylabel('Amplitude')
    
    if not is_jupyter(): plt.savefig(name)

### Produce plots
First plot the original data samples and their corresponding sinc functions that will be used for reconstruction.

In [ ]:
plot_fig(x = data_timepoints, y = data,
         Sincs = True, Sum = False,
         name = 'multirate_sampling_data.pdf')

On a second figure, also plot the reconstructed analogue signal

In [ ]:
plot_fig(x = data_timepoints, y = data,
         Sincs = True, Sum = True,
         name = 'multirate_sampling_reconstructed.pdf')

Finally, show how this can be resampled at a new rate

In [ ]:
plot_fig(x = data_timepoints, y = data,
         Sincs = False, Sum = True,
        name = 'multirate_sampling_resampled.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.